## Mining Polish Bankruptcy Data
<table><tr><td><img src='resources/img_0.jpg'></td><td><img src='resources/img_1.jpg'></td><td><img src='resources/img_2.jpg'></td></tr></table>

### A. Import libraries and data

#### (i) libraries

In [6]:
# general utilities
import pandas as pd
import numpy as np
from scipy.io import arff

# classification models
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

#### (ii) data
- loading data from arff file<br>
Reference: https://discuss.analyticsvidhya.com/t/loading-arff-type-files-in-python/27419

In [14]:
# import data
years = [1,2,3,4,5]
data = {} # dictionary of dataframe

for year in years:
    arr = arff.loadarff(f"data/year{year}.arff")
    data[f"year{str(year)}"] = pd.DataFrame(arr[0])

In [18]:
# check that the data is correctly load
data["year1"]

Attr1    Attr2    Attr3    Attr4     Attr5     Attr6     Attr7  \
0     0.200550  0.37951  0.39641  2.04720   32.3510  0.388250  0.249760   
1     0.209120  0.49988  0.47225  1.94470   14.7860  0.000000  0.258340   
2     0.248660  0.69592  0.26713  1.55480   -1.1523  0.000000  0.309060   
3     0.081483  0.30734  0.45879  2.49280   51.9520  0.149880  0.092704   
4     0.187320  0.61323  0.22960  1.40630   -7.3128  0.187320  0.187320   
...        ...      ...      ...      ...       ...       ...       ...   
7022  0.018371  0.47410 -0.13619  0.60839  -18.4490  0.018371  0.018371   
7023 -0.013359  0.58354 -0.02265  0.92896  -42.2320 -0.013359 -0.015036   
7024  0.006338  0.50276  0.43923  1.87360    9.7417  0.006338  0.012022   
7025 -0.041643  0.84810 -0.12852  0.57485 -121.9200  0.000000 -0.036795   
7026  0.014946  0.94648  0.03211  1.03630  -20.5810  0.000000  0.015260   

         Attr8    Attr9    Attr10  ...    Attr56    Attr57   Attr58    Attr59  \
0     1.330500  1.13890  0.504940  ...  0.121960  0.397180  0.87804  0.001924   
1     0.996010  1.69960  0.497880  ...  0.121300  0.420020  0.85300  0.000000   
2     0.436950  1.30900  0.304080  ...  0.241140  0.817740  0.76599  0.694840   
3     1.866100  1.05710  0.573530  ...  0.054015  0.142070  0.94598  0.000000   
4     0.630700  1.15590  0.386770  ...  0.134850  0.484310  0.86515  0.124440   
...        ...      ...       ...  ...       ...       ...      ...       ...   
7022  0.972030  1.01210  0.460840  ...  0.011909  0.039866  0.98809  0.274140   
7023  0.562890  0.98904  0.328470  ... -0.011082 -0.040671  1.01110  0.805920   
7024  0.983560  1.00830  0.494490  ...  0.008258  0.012817  0.99174  0.000000   
7025  0.179010  0.42138  0.151820  ... -0.232720 -0.274290  0.98788  3.593100   
7026  0.056357  2.96940  0.053341  ...  0.015705  0.280210  0.97443  1.179200   

       Attr60   Attr61   Attr62   Attr63    Attr64  class  
0      8.4160   5.1372   82.658   4.4158   7.42770   b'0'  
1      4.1486   3.2732  107.350   3.4000  60.98700   b'0'  
2      4.9909   3.9510  134.270   2.7185   5.20780   b'0'  
3      4.5746   3.6147   86.435   4.2228   5.54970   b'0'  
4      6.3985   4.3158  127.210   2.8692   7.89800   b'0'  
...       ...      ...      ...      ...       ...    ...  
7022  73.5050  79.2370   31.268  11.6730   5.14890   b'1'  
7023  10.5990   7.1740   94.092   3.8792   1.75720   b'1'  
7024  10.4700   6.0759   51.019   7.1542  62.00100   b'1'  
7025  39.7030   3.1420  261.850   1.3939   0.51005   b'1'  
7026  15.0360   4.1741  108.640   3.3599  35.11800   b'1'  

[7027 rows x 65 columns]

### B. Data preparation and exploration

#### (i) inspecting missing values and data type of data
Using year1 data as an example: <br>
- we can observe that the problem of missing values is quite serious, e.g., for Attr37, number of missing values is 2740
- all attributes are in floating point data type, which is desirable


In [19]:
data["year1"].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7027 entries, 0 to 7026
Data columns (total 65 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Attr1   7024 non-null   float64
 1   Attr2   7024 non-null   float64
 2   Attr3   7024 non-null   float64
 3   Attr4   6997 non-null   float64
 4   Attr5   7019 non-null   float64
 5   Attr6   7024 non-null   float64
 6   Attr7   7024 non-null   float64
 7   Attr8   7002 non-null   float64
 8   Attr9   7026 non-null   float64
 9   Attr10  7024 non-null   float64
 10  Attr11  6988 non-null   float64
 11  Attr12  6997 non-null   float64
 12  Attr13  7027 non-null   float64
 13  Attr14  7024 non-null   float64
 14  Attr15  7025 non-null   float64
 15  Attr16  7002 non-null   float64
 16  Attr17  7002 non-null   float64
 17  Attr18  7024 non-null   float64
 18  Attr19  7027 non-null   float64
 19  Attr20  7027 non-null   float64
 20  Attr21  5405 non-null   float64
 21  Attr22  7024 non-null   float64
 22  

### C. Data modeling

We are performing a classification task, and our team will try out the following models:
- Decision Tree Model
- Random Forest Model
- Gaussian Naive Bayes Model
- Logistic Regression Model

### D. Model evaluation